<a href="https://colab.research.google.com/github/waltz2u/bd/blob/master/Movie_Rental_with_Spark_DataFrames_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Drive mounting

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

## Setting things up

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 https://developer.download.nvi

In [0]:
!ls

gdrive	sample_data  spark-2.2.3-bin-hadoop2.7	spark-2.2.3-bin-hadoop2.7.tgz


Switch to Java 8 if it is Java 11 now

In [0]:
!update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
  0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
* 2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2


Double check if it is Java 8 now

In [0]:
!java -version 

openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)


Set some environment variables

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.2.3-bin-hadoop2.7"

## Spark DataFrame

In [0]:
import findspark
findspark.init("/content/spark-2.4.5-bin-hadoop2.7")
from pyspark.sql import SparkSession
# Sets the Spark master URL to connect to, such as "local" to run locally, 
# "local[4]" to run locally with 4 cores, or "spark://master:7077" to run on a Spark standalone cluster.
spark = SparkSession.builder.master("local[*]").getOrCreate() #

Reading Data

In [0]:
payment_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/payment.csv')
payment_df = payment_df.alias('payment_df')
payment_df.show(3)

+----------+-----------+--------+---------+------+-------------------+-------------------+
|payment_id|customer_id|staff_id|rental_id|amount|       payment_date|        last_update|
+----------+-----------+--------+---------+------+-------------------+-------------------+
|         1|          1|       1|       76|  2.99|2005-05-25 11:30:37|2006-02-15 22:12:30|
|         2|          1|       1|      573|  0.99|2005-05-28 10:35:23|2006-02-15 22:12:30|
|         3|          1|       1|     1185|  5.99|2005-06-15 00:54:12|2006-02-15 22:12:30|
+----------+-----------+--------+---------+------+-------------------+-------------------+
only showing top 3 rows



In [0]:
rental_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/rental.csv')
rental_df = rental_df.alias('rental_df')
rental_df.show(3)

+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|        1|2005-05-24 22:53:30|         367|        130|2005-05-26 22:04:30|       1|2006-02-15 21:30:53|
|        2|2005-05-24 22:54:33|        1525|        459|2005-05-28 19:40:33|       1|2006-02-15 21:30:53|
|        3|2005-05-24 23:03:39|        1711|        408|2005-06-01 22:12:39|       1|2006-02-15 21:30:53|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
only showing top 3 rows



In [0]:
staff_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/staff.csv')
staff_df = staff_df.alias('staff_df')
staff_df.show(3)

+--------+----------+---------+----------+-------+--------------------+--------+------+--------+--------------------+--------------+
|staff_id|first_name|last_name|address_id|picture|               email|store_id|active|username|            password|   last_update|
+--------+----------+---------+----------+-------+--------------------+--------+------+--------+--------------------+--------------+
|       1|      Mike|  Hillyer|         3|   NULL|Mike.Hillyer@saki...|       1|     1|    Mike|8cb2237d0679ca88d...|2/15/2006 3:57|
|       2|       Jon| Stephens|         4|   NULL|Jon.Stephens@saki...|       2|     1|     Jon|                NULL|2/15/2006 3:57|
+--------+----------+---------+----------+-------+--------------------+--------+------+--------+--------------------+--------------+



In [0]:
inventory_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/inventory.csv')
inventory_df = inventory_df.alias('inventory_df')
inventory_df.show(3)

+------------+-------+--------+-------------------+
|inventory_id|film_id|store_id|        last_update|
+------------+-------+--------+-------------------+
|           1|      1|       1|2006-02-15 05:09:17|
|           2|      1|       1|2006-02-15 05:09:17|
|           3|      1|       1|2006-02-15 05:09:17|
+------------+-------+--------+-------------------+
only showing top 3 rows



In [0]:
store_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/store.csv')
store_df = store_df.alias('store_df')
store_df.show(3)

+--------+----------------+----------+-------------------+
|store_id|manager_staff_id|address_id|        last_update|
+--------+----------------+----------+-------------------+
|       1|               1|         1|2006-02-15 04:57:12|
|       2|               2|         2|2006-02-15 04:57:12|
+--------+----------------+----------+-------------------+



In [0]:
address_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/address.csv')
address_df = address_df.alias('address_df')
address_df.show(3)

+----------+------------------+--------+--------+-------+-----------+-----------+-------------------+
|address_id|           address|address2|district|city_id|postal_code|      phone|        last_update|
+----------+------------------+--------+--------+-------+-----------+-----------+-------------------+
|         1| 47 MySakila Drive|    NULL| Alberta|    300|       null|       null|2014-09-25 22:30:27|
|         2|28 MySQL Boulevard|    NULL|     QLD|    576|       null|       null|2014-09-25 22:30:09|
|         3| 23 Workhaven Lane|    NULL| Alberta|    300|       null|14033335568|2014-09-25 22:30:27|
+----------+------------------+--------+--------+-------+-----------+-----------+-------------------+
only showing top 3 rows



In [0]:
customer_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/customer.csv')
customer_df = customer_df.alias('customer_df')
customer_df.show(3)

+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+
|customer_id|store_id|first_name|last_name|               email|address_id|active|        create_date|        last_update|
+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+
|          1|       1|      MARY|    SMITH|MARY.SMITH@sakila...|         5|     1|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          2|       1|  PATRICIA|  JOHNSON|PATRICIA.JOHNSON@...|         6|     1|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          3|       1|     LINDA| WILLIAMS|LINDA.WILLIAMS@sa...|         7|     1|2006-02-14 22:04:36|2006-02-15 04:57:20|
+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+
only showing top 3 rows



In [0]:
film_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/rental/film.csv')
film_df = film_df.alias('film_df')
film_df.show(3)

+-------+----------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+
|film_id|           title|         description|release_year|language_id|original_language_id|rental_duration|rental_rate|length|replacement_cost|rating|    special_features|        last_update|
+-------+----------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+
|      1|ACADEMY DINOSAUR|A Epic Drama of a...|        2006|          1|                NULL|              6|       0.99|    86|           20.99|    PG|Deleted Scenes,Be...|2006-02-15 05:03:42|
|      2|  ACE GOLDFINGER|A Astounding Epis...|        2006|          1|                NULL|              3|       4.99|    48|           12.99|     G|Trailers,Deleted ...|2006-02-15 05:03:42|
|      3|ADAPTATION HOLES|A As